In [1]:
import pandas

In [2]:
df = pandas.read_csv("C:\\Users\\sante\\Desktop\\мой нлп - Лист1-2.csv", encoding="utf-8")

# тестовые данные
valid = df["title"].isin(['Интерстеллар', "Омерзительная восьмерка", "Тройной форсаж: Токийский дрифт"])
test = df[valid]
del test['title']

# удалили наши данные(оставили данные для обучения)
df = df.loc[df['title'] != "Интерстеллар"]
df = df.loc[df['title'] != "Омерзительная восьмерка"]
df = df.loc[df['title'] != "Тройной форсаж: Токийский дрифт"]
del df['title']

In [3]:
 df.head

<bound method NDFrame.head of                                                   text  label
0    Дон Блут – один из самых известных анимационны...     -1
1    Отрицательных рецензий на этот мультфильм очен...     -1
2    \nЧестно признаюсь: в детстве я если и смотрел...     -1
3    \nВпервые я познакомилась с творением студии '...     -1
4    \nЯ узнала про мультфильм 'Анастасия' на продл...     -1
..                                                 ...    ...
585  Питер Джексон уже не тот...  Пишу как человек,...     -1
586  Ждали-ждали - дождались  Около двух лет назад ...     -1
587  Три часа скукоты  В том, что «Хоббит: Нежданно...     -1
588  что-то случилось с любимым нами Средиземьем  В...     -1
589  Детям фильм смотреть нельзя  Фильм не понравил...     -1

[590 rows x 2 columns]>

In [4]:
def tokenizeWord(word):
    word = tokenizer.tokenize(word)
    if len(word) == 1:
        word = word[0]
    else:
        word = ""
    return word


In [5]:
import pymorphy2
from nltk.tokenize import RegexpTokenizer
morph = pymorphy2.MorphAnalyzer()
tokenizer = RegexpTokenizer(r'\w+')
uniqWords = []
tupleIndex = 0
sentences = []
texts = []
for tuple in df.values:
    tupleIndex+=1
    texts.append(tuple[0])
    words = tuple[0].split()
    wordsList = []
    for word in words:
        wordNF = tokenizeWord(morph.parse(word)[0].normal_form)
        uniqWords.append(wordNF)
        wordsList.append(wordNF)
    sentences.append(wordsList)
tupleIndex = 0
for tuple in test.values:
    tupleIndex+=1
    texts.append(tuple[0])
    words = tuple[0].split()
    wordsList = []
    for word in words:
        wordNF = tokenizeWord(morph.parse(word)[0].normal_form)
        uniqWords.append(wordNF)
        wordsList.append(wordNF)
    sentences.append(wordsList)

In [6]:
from gensim import corpora
dictionary = corpora.Dictionary(sentences)
corpus = [dictionary.doc2bow(sentence) for sentence in sentences]

In [7]:
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [8]:
import gensim
NUM_TOPICS = 10
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=15)
for topic in topics:
    print(topic[1])

0.001*"амели" + 0.001*"мёртвый" + 0.001*"героиня" + 0.001*"вдохновлять" + 0.001*"жён" + 0.001*"и" + 0.001*"феноменальный" + 0.001*"собирать" + 0.001*"не" + 0.001*"треть" + 0.001*"завоевать" + 0.001*"наполнить" + 0.001*"в" + 0.000*"остроумный" + 0.000*"персонажами"
0.000*"периодичность" + 0.000*"отказать" + 0.000*"стильность" + 0.000*"вачивски" + 0.000*"оценки" + 0.000*"страданиях" + 0.000*"окружают" + 0.000*"станут" + 0.000*"классик" + 0.000*"новатор" + 0.000*"вачовски" + 0.000*"и" + 0.000*"в" + 0.000*"не" + 0.000*""
0.001*"счету" + 0.001*"лиза" + 0.001*"тарантиновский" + 0.001*"достоверность" + 0.001*"и" + 0.001*"" + 0.001*"унижать" + 0.001*"размазать" + 0.001*"элита" + 0.001*"явления" + 0.001*"полковник" + 0.001*"конкуренция" + 0.001*"негритят" + 0.001*"временной" + 0.001*"сегодняшний"
0.040*"и" + 0.029*"в" + 0.023*"" + 0.022*"не" + 0.015*"что" + 0.014*"на" + 0.014*"он" + 0.013*"это" + 0.012*"я" + 0.011*"с" + 0.011*"фильм" + 0.009*"но" + 0.009*"весь" + 0.008*"как" + 0.008*"который"
0

In [15]:

from gensim.models import CoherenceModel

coherence_model_lda = CoherenceModel(model=ldamodel, texts=sentences, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



Coherence Score:  0.2949455060712663
